In [ ]:
grains_filename = 'Grains.csv'

# Instructions

In the cell above, change the filename to the one you want to plot, then from the menu bar, select `Cell >> Run All`. It will produce the following plots:

1. Confidence
2. Equivalent strain
3. Normal strain ($\varepsilon_{zz} \neq 0$)
4. Plane strain ($\varepsilon_{xx,yy} \neq 0$)
5. Shear strain ($\gamma_{ij} \neq 0$)
    
Plots 2-5 use

$$
    \epsilon_{eq} = \frac{2}{3} \sqrt{\frac{3 (\varepsilon_{xx}^2 + \varepsilon_{yy}^2 + \varepsilon_{zz}^2)}{2} + \frac{3 (\gamma_{xy}^2 + \gamma_{yz}^2 + \gamma_{zx}^2)}{4}}
$$

where

$$
    \varepsilon_{xx} =  2/3 \epsilon_{xx} - 1/3 \epsilon_{yy} - 1/3 \epsilon_{zz} \\
    \varepsilon_{yy} = -1/3 \epsilon_{xx} + 2/3 \epsilon_{yy} - 1/3 \epsilon_{zz} \\
    \varepsilon_{zz} = -1/3 \epsilon_{xx} - 1/3 \epsilon_{yy} + 2/3 \epsilon_{zz} \\
    \gamma_{ij} = 2 \epsilon_{ij},\ i \neq j
$$

givan a strain matrix,

$$
    \epsilon = \left(\begin{array}{c c c}
        \epsilon_{xx} & \epsilon_{xy} & \epsilon_{xz} \\
        \epsilon_{yx} & \epsilon_{yy} & \epsilon_{yz} \\
        \epsilon_{zx} & \epsilon_{zy} & \epsilon_{zz}\end{array}\right)
$$

In [1]:
%pylab

Using matplotlib backend: MacOSX
Populating the interactive namespace from numpy and matplotlib


In [2]:
import numpy as np
from matplotlib import pyplot as plt
from hedm import plot

# Read grains.csv

In [3]:
grains = np.genfromtxt(grains_filename, skip_header=8, names=True, dtype=None)

In [4]:
xyz = grains[np.array(['X', 'Y', 'Z'])].view(np.float64).reshape((-1, 3))
Mc = grains[np.array([
            'O11', 'O12', 'O13',
            'O21', 'O22', 'O23',
            'O31', 'O32', 'O33'])].view(np.float64).reshape((-1, 3, 3))
efab = grains[np.array([
            'eFab11', 'eFab12', 'eFab13',
            'eFab21', 'eFab22', 'eFab23',
            'eFab31', 'eFab32', 'eFab33'])].view(np.float64).reshape((-1, 3, 3))

In [5]:
# Transform from MIDAS coordinates to loading coordinates
#   1. +90 degree rotation about MIDAS X
#   2. +45 degree rotation about new Z
# 90 degree
q = np.radians(90)
c, s = np.cos(q), np.sin(q)
Rx = np.matrix([
        [1, 0, 0],
        [0, c, s],
        [0, -s, c]])
# 45 degree
q = np.radians(45)
c, s = np.cos(q), np.sin(q)
Rz = np.matrix([
        [c, s, 0],
        [-s, c, 0],
        [0, 0, 1]])
# to Loading from MIDAS
Lm = Rz*Rx

In [6]:
# transform XYZ
xyz = np.array([x*(Lm.T) for x in xyz]).view(np.float64).reshape((-1, 3))
# transform orientation matrix, Mc
Lc = np.array([Lm*mat for mat in Mc])
# transform strain
efab = np.array([Lm*e for e in efab])

In [7]:
# equivalent (von Mises) strain
exx =  2./3.*efab[:, 0, 0] - 1./3.*efab[:, 1, 1] - 1./3.*efab[:, 2, 2]
eyy = -1./3.*efab[:, 0, 0] + 2./3.*efab[:, 1, 1] - 1./3.*efab[:, 2, 2]
ezz = -1./3.*efab[:, 0, 0] - 1./3.*efab[:, 1, 1] + 2./3.*efab[:, 2, 2]
exy = 2*efab[:, 0, 1]
eyz = 2*efab[:, 1, 2]
ezx = 2*efab[:, 2, 0]
equiv  = 2./3.*np.sqrt(3*(exx**2 + eyy**2 + ezz**2)/2. + 3*(exy**2 + eyz**2 + ezx**2)/4.)
normal = 2./3.*np.sqrt(3*(  0**2 +   0**2 + ezz**2)/2.)
plane  = 2./3.*np.sqrt(3*(exx**2 + eyy**2 +   0**2)/2.)
shear  = 2./3.*np.sqrt(3*(exy**2 + eyz**2 + ezx**2)/4.)

# Plot

In [8]:
x, y, z = xyz.T

# #fig = plt.figure()
# left = plt.subplot2grid((3, 4), (0, 0), colspan=3, rowspan=3, projection='hedm')
# top  = plt.subplot2grid((3, 4), (0, 3), projection='hedm')
# mid  = plt.subplot2grid((3, 4), (1, 3), projection='hedm')
# bot  = plt.subplot2grid((3, 4), (2, 3), projection='hedm')

plt.rc('text', usetex=True)
plt.rc('font', family='serif')
plt.ion()

figsize=(12,8)

# plot confidence
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='hedm')
pc = ax.grains3D(x, y, z, Lc, c=grains['Confidence'], s=grains['GrainRadius'], crystal='hexagonal')
cb = plt.colorbar(pc, ax=ax)
ax.set_xlabel('X ($\mathrm{\mu m}$)')
ax.set_ylabel('Y ($\mathrm{\mu m}$)')
ax.set_zlabel('Z ($\mathrm{\mu m}$)')
cb.set_label('confidence (Fable-sense)')
ax.set_title('Grain Confidence')

# plot equivalent strain
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='hedm')
pc = ax.grains3D(x, y, z, Lc, c=equiv, s=grains['GrainRadius'], crystal='hexagonal')
cb = plt.colorbar(pc, ax=ax)
ax.set_xlabel('X ($\mathrm{\mu m}$)')
ax.set_ylabel('Y ($\mathrm{\mu m}$)')
ax.set_zlabel('Z ($\mathrm{\mu m}$)')
cb.set_label('equivalent strain ($\mathrm{\mu \epsilon}$)')
ax.set_title('Equivalent Strain')

# plot normal strain
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='hedm')
pc = ax.grains3D(x, y,z, Lc, c=normal, s=grains['GrainRadius'], crystal='hexagonal')
cb = plt.colorbar(pc, ax=ax)
ax.set_xlabel('X ($\mathrm{\mu m}$)')
ax.set_ylabel('Y ($\mathrm{\mu m}$)')
ax.set_zlabel('Z ($\mathrm{\mu m}$)')
cb.set_label('normal strain ($\mathrm{\mu \epsilon}$)')
ax.set_title('Normal Strain')

# plot plane strain
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='hedm')
pc = ax.grains3D(x, y,z, Lc, c=plane, s=grains['GrainRadius'], crystal='hexagonal')
cb = plt.colorbar(pc, ax=ax)
ax.set_xlabel('X ($\mathrm{\mu m}$)')
ax.set_ylabel('Y ($\mathrm{\mu m}$)')
ax.set_zlabel('Z ($\mathrm{\mu m}$)')
cb.set_label('in-plane ($\mathrm{\mu \epsilon}$)')
ax.set_title('Plane Strain')

# plot shear
fig = plt.figure(figsize=figsize)
ax = fig.add_subplot(111, projection='hedm')
pc = ax.grains3D(x, y,z, Lc, c=shear, s=grains['GrainRadius'], crystal='hexagonal')
cb = plt.colorbar(pc, ax=ax)
ax.set_xlabel('X ($\mathrm{\mu m}$)')
ax.set_ylabel('Y ($\mathrm{\mu m}$)')
ax.set_zlabel('Z ($\mathrm{\mu m}$)')
cb.set_label('shear strain ($\mathrm{\mu \epsilon}$)')
ax.set_title('Shear Strain')

plt.draw()
plt.show()